# Name entities recognition on each intent of the dataset

In [2]:
dyn.load(paste0(system2('/usr/libexec/java_home', stdout = TRUE), '/jre/lib/server/libjvm.dylib'))
library(NLP)
library(openNLP)
library(RWeka)
library(rJava)
library(magrittr)
library(stringr)

In [3]:
#install.packages("openNLPmodels.en",
#                 repos = "http://datacube.wu.ac.at/",
#                 type = "source")

In [4]:
citations <- read.csv(file="opinions_sample_after_scraping.csv", header=TRUE, sep=";")

In [5]:
head(citations)

X,case_cite,parties,docket,court,date,intent,id


In [6]:
citation <- citations[66,]
#citation <- "Carousel is the second musical by the team of Richard Rodgers (music) and Oscar Hammerstein II (book and lyrics). The 1945 work was adapted from Ferenc Molnár's 1909 play Liliom, transplanting its Budapest setting to the Maine coastline. The story revolves around carousel barker Billy Bigelow, whose romance with millworker Julie Jordan comes at the price of both their jobs. He participates in a robbery to provide for Julie and their unborn child; after it goes tragically wrong, he is given a chance to make things right. A secondary plot line deals with millworker Carrie Pipperidge and her romance with ambitious fisherman Enoch Snow. The show includes the well-known songs \"If I Loved You\", \"June Is Bustin\' Out All Over\" and \"You'll Never Walk Alone\". Richard Rodgers later wrote that Carousel was his favorite of all his musicals."
intent <- paste(citation$intent, collapse = "")
#intent <- as.String(citation)
intent

[1] "['Appeal from the C. A. 6th Cir.', 'Certiorari denied. Reported below: 306 F. 3d 279.']"

In [7]:
word_ann <- Maxent_Word_Token_Annotator()
sent_ann <- Maxent_Sent_Token_Annotator()

In [11]:
words = c("judgment is reversed", "motion", "motions", "denied", "affirmed", "dismissed", "granted")
citations$outcome <- NA  
for (row in 1:nrow(citations)) {
    indent <- citations[row,"intent"]
    indent_annotations <- annotate(indent, list(sent_ann, word_ann))
    indent_doc <- AnnotatedPlainTextDocument(indent, indent_annotations)
    for (element in sents(indent_doc)) {
        for (word in words) {
            sentence <- str_to_lower(paste(element, collapse = " "))
            if (any(grepl(word, sentence))) {
                outcome <- sentence %>% str_replace_all("[[:punct:]]", " ") %>% str_replace_all("\\s+", 
                  " ")
                citations[row,"outcome"] = outcome
            }
        }
    }
}


# words(intent_doc) %>% head(10)

In [13]:
tail(citations)

X    case_cite          
4650 4994 ['543']            
4651 4995 ['543']            
4652 4996 ['540']            
4653 4997 ['540']            
4654 4998 ['135', '10', '34']
4655 4999 ['540']            
     parties                                                                                  
4650 ['IN RE VOVAK.']                                                                         
4651 ['CULOTTAv.UNITED STATES.']                                                              
4652 ['MILLER ET UX.v.SILVER ET AL.']                                                         
4653 ['EVANSv.FRANKLIN COUNTY COURT OF COMMON PLEAS OF OHIO, DIVISION OF DOMESTIC RELATIONS.']
4654 ['WILLARDv.WOOD.']                                                                       
4655 ['DIAZv.UNITED STATES.']                                                                 
     docket           court                               date                 
4650 ['No. 04-7300.'] ['Supreme Court of United States.'] ['January 10, 2005.']
4651 ['No. 04-6542.'] ['Supreme Court of United States.'] ['November 1, 2004.']
4652 ['No. 02-1687.'] ['Supreme Court of United States.'] ['October 6, 2003.'] 
4653 ['No. 03-6842.'] ['Supreme Court of United States.'] ['December 8, 2003.']
4654 []               []                                  ['May 5, 1890.']     
4655 ['No. 03-5441.'] ['Supreme Court of United States.'] ['October 6, 2003.'] 
     intent                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
person_ann <- Maxent_Entity_Annotator(kind = "person")
location_ann <- Maxent_Entity_Annotator(kind = "location")
organization_ann <- Maxent_Entity_Annotator(kind = "organization")
date_ann <- Maxent_Entity_Annotator(kind = "date")
#misc_ann <- Maxent_Entity_Annotator(kind = "misc")


In [ ]:
intent_annotations <- annotate(intent, list(sent_ann, word_ann, person_ann, location_ann, organization_ann, date_ann))
intent_doc <- AnnotatedPlainTextDocument(intent, intent_annotations)

In [ ]:
entities <- function(doc, kind) {
  s <- doc$content
  a <- annotations(doc)[[1]]
  if(hasArg(kind)) {
    k <- sapply(a$features, `[[`, "kind")
    s[a[k == kind]]
  } else {
    s[a[a$type == "entity"]]
  }
}

In [ ]:
entities(intent_doc, kind = "person")
entities(intent_doc, kind = "location")
entities(intent_doc, kind = "organization")
entities(intent_doc, kind = "date")